In [1]:
import os, sys, pickle
import numpy as np
import pandas as pd
import product_fem as pf
import fenics
import json
from PIL import Image
from scipy import interpolate
import matplotlib.pyplot as plt

# Parameters for inference

Here are things we might want to adjust.

In [2]:
mesh_n = 5
penalty = 1.0
epsilon = 0.03

# Read in the data

This reads in:
1. spatial statistics (e.g., average density at a grid of points), from `.spstats.csv`
2. sampling locations and genetic diversities there, from `.stats.csv`
3. pairwise divergences, from `.pairstats.csv`,
4. the parameters, from `params.json`

and also loads the maps used to parameterize the simulation, as images.

In [ ]:
# load spatial and genetic data
simbase = "density_saddle/out_2059675351901"
basename = "density_saddle/out_2059675351901_stats/rep876970"
basedir = os.path.split(basename)[0]
mapfile = f"{simbase}.spstats.csv"
with open(os.path.join(basedir, "params.json"), "r") as pfile:
    params = json.load(pfile)
    for k in params:
        if len(params[k]) == 1:
            params[k] = params[k][0]
spatial_data = pd.read_csv(f"{basename}.stats.csv", index_col=0)
genetic_data = pd.read_csv(f"{basename}.pairstats.csv", index_col=0)
bias_map = Image.open(params['BIAS_MAP_FILE'])
cov_map = Image.open(params['COVARIANCE_MAP_FILE'])
habitat_map = Image.open(params['HABITAT_MAP_FILE'])
empirical_maps = pd.read_csv(mapfile)
width, height = bias_map.width / params['MAP_RESOLUTION'], bias_map.height / params['MAP_RESOLUTION']
aspect_ratio = width/height
size = (width + height)/2

In [ ]:
params

Here are the images used to parameterize the simulation.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 9))
for (im, lab), ax in zip(((bias_map, 'bias'), (cov_map, 'cov'), (habitat_map, 'habitat')), axes):
    ax.imshow(im, extent=(0, width, 0, height))
    ax.set_title(lab)

# Empirical maps

Here are the maps of average density, fecundity, and establishment
that SLiM recorded during the simulation.
First some helper functions to pull the information out of the pandas table it is in.

In [3]:
def make_map_array(sp, n):
    xvals = np.unique(sp['x'])
    yvals = np.unique(sp['y'])
    nr, nc = len(xvals), len(yvals)
    X = sp['x'].to_numpy().reshape((nr, nc))
    for i in range(nr):
        assert len(np.unique(X[i,:])) == 1 and X[i,0] == xvals[i]
    Y = sp['y'].to_numpy().reshape((nr, nc))
    for j in range(nc):
        assert len(np.unique(Y[:,j])) == 1 and Y[0, j] == yvals[j]
    return xvals, yvals, empirical_maps[n].to_numpy().reshape((nr, nc))

def make_map_fn(sp, n):
    xvals, yvals, Z = make_map_array(sp, n)
    return interpolate.RegularGridInterpolator((xvals, yvals), Z)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 9))
for lab, ax in zip(("density", "fecundity", "establishment"), axes):
    _, _, im = make_map_array(empirical_maps, lab)
    ax.imshow(im, extent=(0, width, 0, height))
    ax.set_title(lab)

Here is some of the data! In this map, circle size is proportional to heterozygosity.

In [ ]:
het_scale = 10**(2-np.floor(np.log10(np.max(spatial_data['het']))))
fig, ax = plt.subplots()
ax.imshow(habitat_map, extent=(0, width, 0, height))
ax.scatter(spatial_data['x'], spatial_data['y'], s=spatial_data['het']*het_scale, c='pink', edgecolors='black')
ax.set_aspect(1.0)
ax.set_title(f"heterozygosity");

# Inference

Now, we'll set up for inference: making a mesh,
and normalizing coordinates and data.

In [ ]:
scaling = {
    'xy': size,
}

mesh = fenics.RectangleMesh(fenics.Point(0.0, 0.0), fenics.Point(width/scaling['xy'], height/scaling['xy']),
                            int(aspect_ratio * mesh_n), mesh_n)
V = fenics.FunctionSpace(mesh, 'CG', 1)
W = pf.ProductFunctionSpace(V)

In [ ]:
fenics.plot(mesh);

In [ ]:
# normalize dxy
train_dxy = genetic_data['dxy'].to_numpy()
scaling['dxy'] = train_dxy.max()
train_dxy /= scaling['dxy']

In [ ]:
# pairwise coordinates needed for kernel density estimate
points = spatial_data[['x', 'y']].to_numpy() / scaling['xy']

def coords_to_pairs(points):
    n = len(points)
    N = int(n * (n + 1) / 2)
    xs = np.zeros((N, 2))
    ys = np.zeros((N, 2))
    x1, y1, x2, y2 = [], [], [], []
    for i in range(n):
        for j in range(i, n):
            x1.append(points[:,0][i])
            y1.append(points[:,1][i])
            x2.append(points[:,0][j])
            y2.append(points[:,1][j])

    xs[:,0] = x1
    xs[:,1] = y1
    ys[:,0] = x2
    ys[:,1] = y2
    return xs, ys

We get the boundary conditions as follows.
To estimate $\pi(x)$ for $x \in \mathbb{R}^2$ let for $i \le j$
$$ w_{ij} = \exp\left( - (|x-x_i|^2 + |x_j - x|^2) / 2 \sigma^2_e - |x_i - x_j|^2 / 2 \epsilon^2 \right), $$
where $\sigma_e$ is four times the mean second-nearest-neighbor distance and $\epsilon$ is a parameter.
Then, using the diverences and diversities $d(i, j)$,
get
$$ \hat \pi(x) = \frac{ \sum_{i \le j} d(i, j) w_{ij} }{ \sum_{i \le j} w_{ij} } . $$
Then, for a point $(x, y) \in \mathbb{R}^2 \times \mathbb{R}^2$,
let
$$ \hat d(x, y) = \hat \pi((x+y)/2) . $$

**Note:** the convergence of the solver below can depend on these parameters!
TODO: figure out how to adjust them.

In [ ]:
dists = np.sqrt(np.subtract.outer(points[:,0], points[:,0])**2 + np.subtract.outer(points[:,1], points[:,1])**2)
sigma_e = 4 * np.mean(dists[np.arange(20), dists.argsort()[:,2]]) # third-largest, row-wise
xs, ys = coords_to_pairs(points)
def kernl(x, y):
    x = np.add(x, y) / 2
    def _dists(x_i, x_j, x):
        diffs = (x_i - x_j) / epsilon, (x_i - x) / sigma_e, (x_j - x) / sigma_e
        dists = np.sum([np.hypot(*xx) for xx in diffs])
        return dists
    dists = np.array([_dists(x_i, y_i, x) for x_i, y_i in zip(xs, ys)])
    scale = 2 / scaling['xy']**2
    return np.exp(-dists / scale)

def boundary(x, y):
    k = kernl(x, y)
    return (train_dxy * k).sum() / k.sum()

In [ ]:
sigma_e

Now we set up and solve the equation.

In [ ]:
eqn = pf.HittingTimes(W, boundary, epsilon=epsilon)
control = eqn.control

# loss functionals
reg = {'l2': [100*penalty, 100*penalty], 'smoothing': [penalty, penalty]}
train_sd = pf.SpatialData(train_dxy, points, W)
train_loss = pf.LossFunctional(train_sd, control, reg)

invp = pf.InverseProblem(eqn, train_loss)

In [ ]:
options = {'ftol': 1e-8, 
           'gtol': 1e-8, 
           'maxcor': 15,
           'maxiter': 100}
m_hats, losses, results = invp.optimize(control, 
                                        method='L-BFGS-B', 
                                        options=options)

In [ ]:
control.update(m_hats[-1])
u_hat = eqn.solve()

Here is an example slice of the 4d solution:

In [ ]:
fenics.plot(u_hat.get_slice((0.25, 0.5)));

And, finally, here are the inferred ellipse and vector fields
for (reverse-time, effective) dispersal:

In [ ]:
ax0, ax1 = eqn.plot_control()
ax0.set_title("inferred sigma")
ax1.set_title("inferred bias");
Q = ax1.collections[0]
ax1.quiverkey(Q, 0.9, 0.7, .002, label=f"{.002}", angle=0);

# Comparison to the (reverse-time) truth

Now we'll compute the "truth" as expected based on the recorded maps,
which we first need to project into our function space.

In [ ]:
density_fn = make_map_fn(empirical_maps, 'density')
density = pf.transforms.callable_to_Function(lambda x, y: density_fn([[x * size, y * size]]), V)
fenics.plot(density).axes.set_title("empirical density");jjb

In [ ]:
establishment_fn = make_map_fn(empirical_maps, 'establishment')
establishment = pf.transforms.callable_to_Function(lambda x, y: establishment_fn([[x * size, y * size]]), V)
fenics.plot(establishment).axes.set_title("empirical establishment");

In [ ]:
fecundity_fn = make_map_fn(empirical_maps, 'fecundity')
fecundity = pf.transforms.callable_to_Function(lambda x, y: fecundity_fn([[x * size, y * size]]), V)
fenics.plot(fecundity).axes.set_title("empirical fecundity");

In [ ]:
log_total_fecundity = density.copy()
log_total_fecundity.vector()[:] = np.log(0.1 + density.vector()[:] * fecundity.vector()[:])
grad_log_total_fecundity = fenics.project(fenics.grad(log_total_fecundity))
fenics.plot(grad_log_total_fecundity).axes.set_title("grad log total fecundity");

In [ ]:
def rgb_to_floats(x, min=-1, max=1):
    """ The "inverse" of floats_to_rgb. Note the denominator is 255, mirroring SLiM. """
    out = min + (max - min) * np.array(x).astype('float') / 255
    return out

def image_fn(im, x, y, min=-1, max=1):
    u = x * params['MAP_RESOLUTION'] * (1 - 1/im.width)
    v = (im.height - y * params['MAP_RESOLUTION']) * (1 - 1/im.height)
    z = im.getpixel((u, v))
    return rgb_to_floats(z, min=min, max=max)

In [ ]:
bias = pf.transforms.vectorized_fn(V, dim=2, name='bias')
bias.vector()[:] = pf.transforms.callable_to_array(
    lambda x, y: params['BIAS'] * image_fn(bias_map, x * size, y * size)[:2],
    V,
).flatten()

In [ ]:
fenics.plot(bias).axes.set_title("forwards-time bias");

In [ ]:
true_bias = fenics.project(2 * establishment * fecundity * (grad_log_total_fecundity - bias))
fenics.plot(true_bias).axes.set_title("true (reverse-time) bias");